In [38]:
import numpy as np
from dataclasses import dataclass
from IPython.display import display, Latex
import sympy

# **Plane Answers to Complex Questions: The Theory of Linear Models, Fifth Ed., Ronald Christensen**

$$\mathbf{Y}=\mathbf{X}\beta+\mathbf{e}$$

where $\mathbf{Y}$ is an $n\times1$ vector of random obesrvations, $\mathbf{X}$ is an $n\times p$ matrix of knwon constants called the model (or design) matrix, $\beta$ is a $p\times1$ vector of unobservable fixed parameters, and $\mathbf{e}$ is an $n\times1$ vector of unobservable random errors. Both $\mathbf{Y}$ and $\mathbf{e}$ are random vectors.

In [60]:

class LinearModel:

    def __init__(self, 
                 n: int, 
                 p: int, 
                 Y: np.array=None, 
                 X: np.array=None, 
                 Beta: np.array=None, 
                 e: np.array=None
                 ):
        self.n = n
        self.p = p
        self.Y = Y
        self.X = X
        self.Beta = Beta
        self.e = e
        if self.Y is not None and self.Y.shape != (self.n, 1):
            display(Latex(f'$$Y\\text{{ must be of shape ({self.n}, 1), not {self.Y.shape}}}$$'))
            raise ValueError()
        
        if self.X is not None and self.X.shape != (self.n, self.p):
            display(Latex(f'$$X\\text{{ must be of shape }}(n={self.n},p={self.p})\\text{{, not }}{self.X.shape}$$'))
            raise ValueError()
        
        if self.Beta is not None and self.Beta.shape != (self.n, 1):
            display(Latex(f'$$\\beta\\text{{ must be of shape ({self.p},1), not {self.Beta.shape}}}$$'))
            raise ValueError()
        
        if self.e is not None and self.e.shape != (self.n, 1):
            display(Latex(f'$$e\\text{{ must be of shape ({self.n},1), not {self.e.shape}}}$$'))
            raise ValueError()
        
    def display(self):
        if self.Y is None:
            Y = [[sympy.Symbol(f"y_{{{i + 1}}}")] for i in range(self.n)]
        else:
            Y = self.Y
        
        if self.X is None:
            if self.p > 1:
                X = [[sympy.Symbol(f'x_{{{i + 1},{j + 1}}}') for j in range(self.p)] for i in range(self.n)]
            else:
                X = [[sympy.Symbol(f'x_{{{i + 1}}}')] for i in range(self.n)]
        else: 
            X = self.X

        if self.Beta is None:
            Beta = [[sympy.Symbol(f'\\beta_{{{i + 1}}}')] for i in range(self.p)]
        else:
            Beta = self.Beta
        
        if self.e is None:
            e = [[sympy.Symbol(f'e_{{{i + 1}}}')] for i in range(self.n)]
        else:
            e = self.e

        Y = sympy.Matrix(Y)._repr_latex_().replace('\\displaystyle ', '').replace('$', '')
        X = sympy.Matrix(X)._repr_latex_().replace('\\displaystyle ', '').replace('$', '')
        Beta = sympy.Matrix(Beta)._repr_latex_().replace('\\displaystyle ', '').replace('$', '')
        e = sympy.Matrix(e)._repr_latex_().replace('\\displaystyle ', '').replace('$', '')


        display(Latex(f"$${Y}={X}{Beta}+{e}$$"))



*Linear models specify the expected value of the observed data $\mathbf{Y}$ as a linear function of the parameter vector $\beta$*. To be a *linear model* $\mathbb{E}(\mathbf{e})=0$. The *standard linear model* assumes that the errors have a common unknown variance and $Cov(\mathbf{e})=\sigma^2\mathbf{I}$ where $\sigma^2$ is some unknown parameter.

In order to get tests and confidence regions for a standard linear model, we will assume that the $\mathbf{e}\sim\mathcal{N}(\mathbf{0},\sigma^2\mathbf{I},)$.

The essence of linear model theory is decompose the observations $\mathbf{Y}$ into $\mathbf{Y}=\hat{\mathbf{Y}}+\hat{\mathbf{e}}$. Here $\hat{\mathbf{Y}}$ is a vector of *fitted values* that contains all the information for estimating the unknown parameter vector $\beta$, where $\hat{\mathbf{Y}}\in C(\mathbf{X})$. With any good statistical procedure, it is necessary to investigate whether the assumtpions that have been made are reasonable. $\hat{\mathbf{e}}$ is a vector of *residuals* that contains the information used for checking the assumptions built into the linear model and for estimating the parameters associated with the errors.

Applications of linear models often fall into two special cases: *Regresion Analysis* and *Analysis of Variance*. Regression refers to models in which the matrix $\mathbf{X}'\mathbf{X}$ is nonsingular. Analysis of Variance (ANOVA) models are models in which the model matrix consists entirely of zeros and ones. ANOVA models are sometimes classification models but in recent years that name has been co-opted for models in which the components of $Y$ are binary.




**Example 1.0.1** - Simple Linear Regression

Consider the model

$$y_i=\beta_0+\beta_1x_i+e_i$$

for $i=1,\ldots,6$, $(x_1,x_2,x_3,x_4,x_5,x_6)=(1,2,3,4,5,6)$, where the $e_i\overset{\text{i.i.d.}}{\sim}\mathcal{N}(0,\sigma^2)$. In matrix notation we can write this as


$$\begin{bmatrix}y_1\\y_2\\y_3\\y_4\\y_5\\y_6\end{bmatrix}=\begin{bmatrix}1&1\\1&2\\1&3\\1&4\\1&5\\1&6\end{bmatrix}\begin{bmatrix}\beta_0\\\beta_1\end{bmatrix}+\begin{bmatrix}e_1\\e_2\\e_3\\e_4\\e_5\\e_6\end{bmatrix}$$
$$Y=X\beta+e$$





In [64]:
class SimpleLinearRegression(LinearModel):
    def __init__(self, 
                 n: int, 
                 Y: np.array=None, 
                 X: np.array=None, 
                 Beta: np.array=None, 
                 e: np.array=None
                 ):
        super(SimpleLinearRegression, self).__init__(
            n=n,
            p=2,
            Y=Y,
            X=X, 
            Beta=Beta,
            e=e
        )


In [63]:
simple_linear_regression = SimpleLinearRegression(
    n=6,
    X=np.hstack((np.ones((6, 1)), np.arange(1, 7).reshape(-1, 1)))
)

simple_linear_regression.display()

<IPython.core.display.Latex object>

**Example 1.0.2** - One-Way Analysis of Variance

$$y_{ij}=\mu+\alpha_i+e_{ij}$$

for $i=1,\ldots,3$, $j=1,\ldots,N$ $(N_1,N_2,N_3)=(3,1,2)$, where $e_{ij}\overset{\text{i.i.d.}}{\sim}\mathcal{N}(0,\sigma^2)$, which can be written as

$$\begin{bmatrix}y_{11}\\y_{12}\\y_{13}\\y_{21}\\y_{31}\\y_{32}\end{bmatrix}=\begin{bmatrix}1&1&0&0\\1&1&0&0\\1&1&0&0\\1&0&1&0\\1&0&0&1\\1&0&0&1\end{bmatrix}\begin{bmatrix}\mu\\\alpha_1\\\alpha_2\\\alpha_3\end{bmatrix}+\begin{bmatrix}e_{11}\\e_{12}\\e_{13}\\e_{21}\\e_{31}\\e_{32}\end{bmatrix}$$


There are a couple of useful alternative methods for writing $\mathbf{Y}=\mathbf{X}\beta+\mathbf{e}$. Write $\mathbf{X}$ in terms of its columns and its rows as

$$\mathbf{X}_{n\times p}=\left[\mathbf{X}_1,\ldots,\mathbf{X}_p\right]=\begin{bmatrix}x_1'\\\vdots\\x_n'\end{bmatrix}$$

This leads to

$$\mathbf{Y}=\sum_{i=1}^p\beta_j\mathbf{X}_j+\mathbf{e}$$

and also, listing the elements of $\mathbf{Y}$ and $\mathbf{e}$ in the obvious way,

$$y_i=x_i'\beta_i+e_i,\qquad i=1,\ldots,n$$



## **Appendix A - Vector Spaces**

**Definition A.1** Let $\mathcal{M}$ be a set, $x,y,z\in\mathcal{M}$ and $\alpha,\beta,\xi$ be scalars. $\mathcal{M}$ is a *vector space* if

1. $\forall x.\forall y.\forall z:(x+y)+z=x+(y+z)$.

2. $\forall x.\forall y.\forall z:x+y=y+x$.

3. $\exists y.\forall x:x+0=x=0+x$ ($y=0$).

4. $\forall x.\exists y: x+y=0=y+x$ ($y=-x$).

5. $\forall\alpha.\forall x.\forall y:\alpha(x+y)=\alpha x+\alpha y$.

6. $\forall\alpha.\forall\beta.\forall x:(\alpha+\beta)x=\alpha x+\beta x$.

7. $\forall\alpha.\forall\beta.\forall x:(\alpha\beta)x=\alpha(\beta x)$.

8. $\exists \xi.\forall x:\xi x=x$ ($\xi=1$).


**Definition A.2** Let $\mathcal{M}$ be a vector space, and let $\mathcal{N}$ be a set with $\mathcal{N}\subset\mathcal{M}$. $\mathcal{N}$ is a *subspace* of $\mathcal{M}$ if $\mathcal{N}$ is a vector space using the same defintions of vector addition and scalar multiplication as for $\mathcal{M}$.


Thinking of vectors in three dimensions as $(x,y,z)^T$. The subpsace consisting of the $z$ axis is

$$\left\{\begin{pmatrix}0\\0\\z\end{pmatrix}:z\in\mathbb{R}\right\}$$

The subspace consisting of the $x,y$ plane is 

$$\left\{\begin{pmatrix}x\\y\\0\end{pmatrix}:x,y\in\mathbb{R}\right\}$$

The subpsace consisting of the plane that is perpendicular to the line $x=y$ in the $x,y$ plane is

$$\left\{\begin{pmatrix}x\\-x\\z\end{pmatrix}:x,z\in\mathbb{R}\right\}$$

## **Appendix D - Multivariate Distributions**

Let $(x_1,\ldots,x_n)^T$ be a random vector. The joint cumulative distribution function of $(x_1,\ldots,x_n)^T$ is

$$F(u_1,\ldots,u_n)\equiv\text{Pr}\left[x_1\le u_1,\ldots,x_n\le u_n\right]$$

If $F(u_1,\ldots,u_n)$ is the cdf of a discrete random variable, we can define a joint probabilty mass function

$$f(u_1,\ldots,u_n)\equiv\text{Pr}\left[x_1=\mu_1,\ldots,x_n= u_n\right]$$

If $F(u_1,\ldots,u_n)$ admits the $n$-th order mixed partial derivative, then we can define a joint probability density function

$$f(u_1,\ldots,u_n)\equiv\frac{\partial^n}{\partial u_1\ldots\partial u_n}F(u_1,\ldots,u_n)$$

The cdf can be recovered from the density as

$$F(u_1,\ldots,u_n)=\int_{-\infty}^{u_1}\ldots\int_{-\infty}^{u_n}f(w_1,\ldots,w_n)dw_1\ldots dw_n$$

